In [4]:
import pandas as pd
import numpy as np 
import pathlib
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from keras.layers import TimeDistributed
from keras.optimizers import Adam

In [12]:
# Loading the dataset 
data_dir = pathlib.Path('/home/lv11/Documents/ProyectosPython/sentimentAnalysis/train')
nf = pd.read_csv(data_dir / 'tweetsDataset1.csv',skiprows=1,names=['Message','Target'])
#print(nf.head(20))


In [14]:
nlp = English()
stop_words = list(STOP_WORDS)

messages = nf['Message']
labels = nf['Target']
#print(stop_words)

def spacy_tokenizer(sentence):
    tokens = nlp(sentence)
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != '-PRON-' else word.lower_ for word in tokens ]
    tokens = [ word for word in tokens if word not in stop_words and word not in punctuation ]
    return tokens

In [16]:
def vectorization(tweets):
    vector = CountVectorizer(tokenizer=spacy_tokenizer)
    frequency_matrix = vector.fit_transform(tweets)
    sum_frequencies = np.sum(frequency_matrix,axis=0)
    frequency = np.squeeze(np.asarray(sum_frequencies))
    frequency_dataFrame = pd.DataFrame([frequency],columns=vector.get_feature_names()).transpose()
    return frequency_dataFrame

In [18]:
def splitting(messages,labels):
    X_train,X_test,y_train,y_test = train_test_split(messages,labels,test_size=0.25,random_state=42,shuffle=True)
    return X_train,X_test,y_train,y_test

In [ ]:
def tokenization_tweets(dataset,features):
    tokenization = TfidfVectorizer(max_features=features)
    tokenization.fit(dataset)
    dataset_transformed = tokenization.transform(dataset).toarray()
    return dataset_transformed

In [21]:
def train(X_train_mod, y_train, features, shuffle, drop, layer1, layer2, epoch, lr, epsilon, validation):
    model_nn = Sequential()
    model_nn.add(Dense(layer1, input_shape=(features,), activation='relu'))
    model_nn.add(Dropout(drop))
    model_nn.add(Dense(layer2, activation='sigmoid'))
    model_nn.add(Dropout(drop))
    model_nn.add(Dense(3, activation='softmax'))
    
    optimizer = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=epsilon,decay=0.0, amsgrad=False)
    model_nn.compile(loss='sparse_categorical_crossentropy',
                 optimizer=optimizer,
                 metrics=['accuracy'])
    model_nn.fit(np.array(X_train_mod), y_train,
                 batch_size=32,
                 epochs=epoch,
                 verbose=1,
                 validation_split=validation,
                 shuffle=True)
    return model_nn

In [22]:
def test(X_test,model):
    predict = model.predict(X_test)
    return predict

In [23]:
def modelo1(X_train,y_train):
    features = 1554 #1727 # puede ser 3727 o un valor asi que vi como len() del array de X_train 
    shuffle = True
    drop = 0.5
    layer1 = 512
    layer2 = 256
    epoch = 10
    lr = 0.002
    epsilon = 1e-9
    validation = 0.1
    X_train_mod = tokenization_tweets(X_train,features)
    modelo = train(X_train_mod, y_train,features,shuffle,drop,layer1,layer2,epoch,lr,epsilon,validation)
    return modelo 

In [24]:
frequency_dataFrame = vectorization(messages)

In [25]:
X_train,X_test,y_train,y_test = splitting(messages,labels)

In [26]:
modelo = modelo1(X_train,y_train)

Train on 1554 samples, validate on 173 samples
Epoch 1/10
1554/1554 [==============================] - 2s 1ms/step - loss: 0.7193 - accuracy: 0.6171 - val_loss: 0.3311 - val_accuracy: 0.9017
Epoch 2/10
1554/1554 [==============================] - 1s 913us/step - loss: 0.2586 - accuracy: 0.8867 - val_loss: 0.2328 - val_accuracy: 0.8844
Epoch 3/10
1554/1554 [==============================] - 1s 902us/step - loss: 0.1260 - accuracy: 0.9498 - val_loss: 0.2262 - val_accuracy: 0.8960
Epoch 4/10
1554/1554 [==============================] - 1s 885us/step - loss: 0.0753 - accuracy: 0.9736 - val_loss: 0.2772 - val_accuracy: 0.8613
Epoch 5/10
1554/1554 [==============================] - 1s 883us/step - loss: 0.0409 - accuracy: 0.9826 - val_loss: 0.3995 - val_accuracy: 0.8728
Epoch 6/10
1554/1554 [==============================] - 1s 904us/step - loss: 0.0334 - accuracy: 0.9871 - val_loss: 0.3603 - val_accuracy: 0.8728
Epoch 7/10
1554/1554 [==============================] - 1s 876us/step - loss: 0

In [27]:
modelo.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               796160    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 771       
Total params: 928,259
Trainable params: 928,259
Non-trainable params: 0
_________________________________________________________________


In [36]:
input_data = tokenization_tweets(X_test,1554)
input_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [82]:
pred = modelo.predict(input_data)
X_test

941                                   I miss you unhappy 
1298    Thanks for being top engaged community members...
1313    I agree. My issue would be that it all has to ...
1270                this is so sad what the fuck unhappy 
464     The video will definitely be a 30 minute episo...
                              ...                        
1185    this is damn.... theres people who sincerely w...
377     so upsetting the abuse these kids get unhappy ...
535                                             clauditte
892                   base unver ajalah MINAHAES unhappy 
1761                                              so? sad
Name: Message, Length: 576, dtype: object

In [102]:
xtest = [x for x in X_test]
pred[0][1]
for i in range(len(pred)):
    sent = (0 if pred[i][0] <= 0.5 else 1)
    print("tweet = %s, sentiment = %s" % (xtest[i],sent))

eet = The most painful goodbyes are the onesthat are never said and never explained crying  unhappy  :/, sentiment = 0
tweet =  wifi, sentiment = 1
tweet = Just win at weekend and it will be a late gift, sentiment = 0
tweet =  I'm aborting mission now have a nice day though x , sentiment = 1
tweet = Be happy, sentiment = 0
tweet =  w, sentiment = 0
tweet = YG should have sent them to MCD. I want to see them holding the trophy unhappy  anyways .9, sentiment = 1
tweet =  idea of saying No to almo, sentiment = 0
tweet = I can't do it all on my own. (See what I did there? ) .1, sentiment = 1
tweet = I dedicate this song to, sentiment = 0
tweet = wish knock out lang talaga for the new school year are good and cooperative groupmates please unhappy , sentiment = 1
tweet =  you change your whole life happy, sentiment = 0
tweet = CAN SOMEONE PLS GIFT ME XAYAH USING PI PLSSSS? unhappy , sentiment = 0
tweet = Don't be buffalo unhappy , sentiment = 0
tweet = I check Beautiful News every single day